# Freddie Mac: 30-year fixed rate mortgage average

#### Load Python tools and Jupyter config

In [1]:
import os
import us
import json
import requests
import numpy as np
import pandas as pd
import jupyter_black
import altair as alt
import altair_stiles as altstiles
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [3]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

---

## Read data

#### Fredie Mac: US 30-Year [fixed rate mortgage average](https://www.freddiemac.com/pmms)

In [4]:
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
}

In [5]:
url = 'https://www.freddiemac.com/pmms/docs/historicalweeklydata.xlsx?tok=tfAZUEGe'

In [6]:
src = pd.read_excel(url, storage_options=headers, skiprows=6, parse_dates=['Week'], skipfooter=1)[['Week', 'FRM','FRM.1']].sort_values('Week', ascending=True).rename(columns={'Week': 'date', 'FRM': 'value', 'FRM.1': 'value15yr'}).reset_index(drop=True)

In [7]:
df = src[src['date'] > "2000-01-01"].reset_index(drop=True).copy()

In [8]:
df["indicator"] = "US fixed rates mortgage average"
df["source"] = "https://www.freddiemac.com/pmms"

In [9]:
df['fetched'] = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S+00:00")
df['change'] = (df['value'].pct_change(1) * 100).round(2)

In [23]:
latest =  df[df['date'] == df['date'].max()].copy()
date_fetched = pd.to_datetime(latest['fetched']).dt.strftime('%b. %-d, %Y').iloc[0]
date_updated = pd.to_datetime(latest['date']).dt.strftime('%b. %-d, %Y').iloc[0]
current_value = df[df['date'] == df['date'].max()]['value'].iloc[0]
change_from_last_week = df[df['date'] == df['date'].max()]['change'].iloc[0]

In [20]:
date_updated

'Jan. 7, 2000'

---

## Exports

In [12]:
df.to_csv("data/processed/freddie-mac-30-year-fixed-mortgage-rate.csv", index=False)

In [13]:
df.to_json(
    "data/processed/freddie-mac-30-year-fixed-mortgage-rate.json", indent=4, orient="records"
)